In [1]:
import geopandas as gpd
import fiona
from shapely.geometry import LineString

fiona.supported_drivers['LIBKML'] = "rw"

In [2]:
def buffer(gdf, buffer_col_name):
    return(gdf['geometry'].buffer(gdf[buffer_col_name]))


def make_variable_buffer(input_df, buffer_col_name):
    # Copying the input_df
    buff_df = input_df.copy()
    
    # Creating the varying-sized buffer
    buff_df['geometry'] = buff_df.apply(buffer, args= [buffer_col_name], axis= 1)
    return(buff_df)

In [3]:
# Read parsed log file
df = gpd.read_file("parsed_log.csv")

In [4]:
# Convert dataframe to geodataframe and export to .shp 
gdf = gpd.GeoDataFrame(df, geometry= gpd.points_from_xy(df['Longitude'], df['Latitude'], crs= "EPSG:4326"))
gdf['Accuracy'] = gdf['Accuracy'].astype("float64")
gdf.to_file("log_points.shp")

In [5]:
# Get statistics about the accuracy
gdf.describe()

,Accuracy
count,600.000000
mean,2.059185
std,1.020670
min,1.000000
25%,1.816000
50%,1.900000
75%,1.980000
max,11.500000


In [6]:
# Reproject geodataframe to GGRS87
gdf_ggrs87 = gdf.to_crs(crs= "EPSG:2100")

In [7]:
# Create a buffered geodataframe based on accuracy attribute & export it to .shp
buff_gdf = make_variable_buffer(gdf_ggrs87, "Accuracy")
buff_gdf.to_file("buffer_log_points.shp")

In [8]:
# Export WGS84 geodataframe to kml for google earth
gdf.to_file("log_points.kml", driver= "LIBKML")

In [9]:
# Convert GGRS87 geodataframe to line-string
line_string = LineString(gdf_ggrs87['geometry'])
line_gdf = gpd.GeoDataFrame([], columns=["line"], crs= "EPSG:2100", geometry= [line_string])
# line_gdf.to_file("log_line.shp")

In [10]:
# Export line geodataframe to .dxf for ktimanet
line_gdf['geometry'].to_file("log_line.dxf", driver= "DXF")